In [ ]:
#Importing the necessary libraries

from PIL import Image
import numpy as np
import pandas as pd
import os
import glob
import cv2
import matplotlib.pylab as plt

from keras import optimizers
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense

In [ ]:
#Path variables
path_train_normal = r"C:\Users\nikhil\Documents\Datasets\chest_xray\train\NORMAL"
path_train_pnem = r"C:\Users\nikhil\Documents\Datasets\chest_xray\train\PNEUMONIA"

path_val_normal = r"C:\Users\nikhil\Documents\Datasets\chest_xray\val\NORMAL"
path_val_pnem = r"C:\Users\nikhil\Documents\Datasets\chest_xray\val\PNEUMONIA"

path_test_normal = r"C:\Users\nikhil\Documents\Datasets\chest_xray\test\NORMAL"
path_test_pnem = r"C:\Users\nikhil\Documents\Datasets\chest_xray\test\PNEUMONIA"

In [ ]:
#Variables to hold the number of jpegs in the directory

count_jpeg_normal = len(glob.glob1(path_train_normal,"*.jpeg"))
count_jpeg_pneum = len(glob.glob1(path_train_pnem,"*.jpeg"))

count_jpeg_normal_val = len(glob.glob1(path_val_normal,"*.jpeg"))
count_jpeg_pneum_val = len(glob.glob1(path_val_pnem,"*.jpeg"))

count_jpeg_normal_test = len(glob.glob1(path_test_normal,"*.jpeg"))
count_jpeg_pneum_test = len(glob.glob1(path_test_pnem,"*.jpeg"))


In [ ]:
print(f"NORMAL - Training {count_jpeg_normal} \nPNEUMONIA - Training {count_jpeg_pneum} \nNORMAL - Validation {count_jpeg_normal_val} \nPNEUMONIA - Validation {count_jpeg_pneum_val} \nNORMAL - Test {count_jpeg_normal_test} \nPNEUMONIA - Test {count_jpeg_pneum_test}")

In [ ]:
#Defining the width, height and the dimensions - its use will come next, during resizing the images
width = 128
height = 128
dim = (width, height)

In [ ]:
#Collecting the images and resizing - NORMAL (takes a bit)
images_NORMAL = []
dim = (width, height)
for i in range(count_jpeg_normal):
    img = cv2.imread(path_train_normal +"/"+ str(glob.glob1(path_train_normal,"*.jpeg")[i]))
    img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    images_NORMAL.append(img)

In [ ]:
#Shape of images
np.array(images_NORMAL).shape

In [ ]:
#y_NORMAL are the patients that do not have PNEUMONIA
y_NORMAL = np.zeros(np.array(images_NORMAL).shape[0])

In [ ]:
y_NORMAL.shape

In [ ]:
#Collecting the images and resizing - PNEUMONIA (takes a bit more)
images_PNEUMONIA = []
dim = (width, height)
for j in range(count_jpeg_pneum):
    img = cv2.imread(path_train_pnem +"/"+ str(glob.glob1(path_train_pnem,"*.jpeg")[j]))
    img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    images_PNEUMONIA.append(img)

In [ ]:
#Shape of images
np.array(images_PNEUMONIA).shape

In [ ]:
#y_NORMAL are the patients that have PNEUMONIA
y_PNEUMONIA = np.ones(np.array(images_PNEUMONIA).shape[0])

In [ ]:
y_PNEUMONIA.shape

In [ ]:
#Concatenating the NORMAL and PNEUMONIA arrays (independent variables)
X = np.concatenate([np.array(images_NORMAL), np.array(images_PNEUMONIA)], axis = 0)

In [ ]:
del images_NORMAL, images_PNEUMONIA

In [ ]:
#Concatenating the predicted values of NORMAL and PNEUMONIA arrays (dependent variables)
Y = np.concatenate([np.array(y_NORMAL), np.array(y_PNEUMONIA)], axis = 0).T # the .T is to get the transpose of the matrix

In [ ]:
del y_NORMAL, y_PNEUMONIA

In [ ]:
'''from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)'''

In [ ]:
#Variable that holds the number of classes to predict
classes_to_predict = pd.Series(Y.tolist()).value_counts().count()
print(classes_to_predict)

In [ ]:
#Collecting the images and resizing - NORMAL - Validation set (almost instant)
images_NORMAL_validation = []
dim = (width, height)
for k in range(count_jpeg_normal_val):
    img = cv2.imread(path_val_normal +"/"+ str(glob.glob1(path_val_normal,"*.jpeg")[k]))
    img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    images_NORMAL_validation.append(img)

In [ ]:
np.array(images_NORMAL_validation).shape

In [ ]:
Y_val_NORMAL = np.zeros(np.array(images_NORMAL_validation).shape[0])

In [ ]:
#Collecting the images and resizing - PNEUMONIA - Validation set (almost instant too)
images_PNEUMONIA_validation = []
dim = (width, height)
for l in range(count_jpeg_pneum_val):
    img = cv2.imread(path_val_pnem +"/"+ str(glob.glob1(path_val_pnem,"*.jpeg")[l]))
    img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    images_PNEUMONIA_validation.append(img)

In [ ]:
np.array(images_PNEUMONIA_validation).shape

In [ ]:
Y_val_PNEUMONIA = np.ones(np.array(images_PNEUMONIA_validation).shape[0])

In [ ]:
#Concatenating indepedent variables of the validation dataset
X_VAL = np.concatenate([images_NORMAL_validation, images_PNEUMONIA_validation], axis = 0)

In [ ]:
del images_NORMAL_validation, images_PNEUMONIA_validation

In [ ]:
#Concatenating depedent variables of the validation dataset
Y_VAL = np.concatenate([Y_val_NORMAL, Y_val_PNEUMONIA], axis = 0)

In [ ]:
del Y_val_NORMAL, Y_val_PNEUMONIA

In [ ]:
#Building the model

model = Sequential()
model.add(
    Conv2D(
        32, 
        kernel_size = (5, 5), 
        strides = (1, 1), 
        activation = 'relu', 
        input_shape = X.shape[1:]))

model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2)))
          
model.add(
    Conv2D(
        64, 
        (5, 5), 
        activation = 'relu'))

model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Flatten())

model.add(Dense(X.shape[1], activation = "tanh"))
model.add(Dense(1000, activation = "tanh"))
model.add(Dense(1000, activation = "tanh"))
model.add(Dense(1000, activation = "tanh"))
model.add(Dense(1000, activation = "tanh"))
model.add(Dense(1000, activation = "tanh"))
model.add(Dense(1000, activation = "tanh"))
model.add(Dense(1, activation='sigmoid'))

sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss = "binary_crossentropy", optimizer = sgd, metrics = ['accuracy'])


In [ ]:
model.summary()

In [ ]:
#Re-resizing:
X = X.astype('float32')
X_VAL = X_VAL.astype('float32')
X /= 255
X_VAL /= 255

In [ ]:
modelfit = model.fit(
    X,
    Y,
    batch_size = 10,
    epochs = 10,
    verbose = 1,
    validation_data = (X_VAL, Y_VAL))

In [ ]:
modelfit.history.keys()

In [ ]:
max(modelfit.history['acc'])

In [ ]:
plt.plot(modelfit.history['acc'])

In [ ]:
plt.plot(model.history.history['val_acc'])

In [ ]:
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('loss')
plt.plot(model.history.history['acc'], model.history.history['loss'])
plt.show()

In [ ]:
plt.plot(model.history.history['loss'])
plt.plot(model.history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
score = model.evaluate(X_VAL, Y_VAL, verbose = 0)
print(f"Validation loss: {score[0]}, Validation acc: {score[1]}")

In [ ]:
path_test_normal = r"C:\Users\nikhil\Documents\Datasets\chest_xray\test\NORMAL"
path_test_pnem = r"C:\Users\nikhil\Documents\Datasets\chest_xray\test\PNEUMONIA"

count_jpeg_normal_test = len(glob.glob1(path_test_normal,"*.jpeg"))
count_jpeg_pneum_test = len(glob.glob1(path_test_pnem,"*.jpeg"))


In [ ]:
#Collecting the images and resizing - NORMAL - Test set
images_NORMAL_test = []
dim = (width, height)
for i in range(count_jpeg_normal_test):
    img = cv2.imread(path_test_normal +"/"+ str(glob.glob1(path_test_normal,"*.jpeg")[l]))
    img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    images_NORMAL_test.append(img)

In [ ]:
np.asarray(images_NORMAL_test).shape

In [ ]:
#Collecting the images and resizing - PNEUMONIA - Test set
images_PNEUMONIA_test = []
dim = (width, height)
for i in range(count_jpeg_pneum_test):
    img = cv2.imread(path_test_pnem +"/"+ str(glob.glob1(path_test_pnem,"*.jpeg")[l]))
    img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    images_PNEUMONIA_test.append(img)

In [ ]:
np.asarray(images_PNEUMONIA_test).shape

In [ ]:
ypredNORMAL = model.predict_classes(np.array(images_NORMAL_test))

In [ ]:
ypredPNEM = model.predict(np.array(images_PNEUMONIA_test))

In [ ]:
print(yPNEM)

In [ ]:
'''https://machinelearningmastery.com/how-to-make-classification-and-regression-predictions-for-deep-learning-models-in-keras/
https://github.com/adventuresinML/adventures-in-ml-code/blob/master/keras_cnn.py'''